## Importing all necessary libraries

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

import requests # library to handle requests

print('Libraries imported.')

Solving environment: / 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - conda-forge/linux-64::pytorch==1.8.0=cpu_py37hafa7651_0
  - defaults/noarch::ibm-wsrt-py37main-keep==0.0.0=2020
  - defaults/noarch::ibm-wsrt-py37main-main==custom=2020
done

# All requested packages already installed.

Solving environment: - 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - conda-forge/linux-64::pytorch==1.8.0=cpu_py37hafa7651_0
  - defaults/noarch::ibm-wsrt-py37main-keep==0.0.0=2020
  - defaults/noarch::ibm-wsrt-py37main-main==custom=2020
done

# All requested packages already installed.

Libraries imported.


## Reading the GeoJSON file that contains the neighborhoods of Zurich

In [2]:
# The code was removed by Watson Studio for sharing.

In [3]:
zurich_data = pd.read_json(body, orient='values')
zurich_data.head()

,type,name,features
0,FeatureCollection,stzh.adm_vermbezirke_beschr_p,"{'type': 'Feature', 'geometry': {'type': 'Poin..."
1,FeatureCollection,stzh.adm_vermbezirke_beschr_p,"{'type': 'Feature', 'geometry': {'type': 'Poin..."
2,FeatureCollection,stzh.adm_vermbezirke_beschr_p,"{'type': 'Feature', 'geometry': {'type': 'Poin..."
3,FeatureCollection,stzh.adm_vermbezirke_beschr_p,"{'type': 'Feature', 'geometry': {'type': 'Poin..."
4,FeatureCollection,stzh.adm_vermbezirke_beschr_p,"{'type': 'Feature', 'geometry': {'type': 'Poin..."


In [4]:
# Extract features into a dataframe
zurich_data_features = zurich_data['features']
zurich_data_features.head()

0    {'type': 'Feature', 'geometry': {'type': 'Poin...
1    {'type': 'Feature', 'geometry': {'type': 'Poin...
2    {'type': 'Feature', 'geometry': {'type': 'Poin...
3    {'type': 'Feature', 'geometry': {'type': 'Poin...
4    {'type': 'Feature', 'geometry': {'type': 'Poin...
Name: features, dtype: object

In [5]:
# Define the dataframe columns for a neighborhoods dataframe
column_names = ['Borough','Neighborhood', 'Latitude', 'Longitude'] 

# Instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

In [6]:
neighborhoods

,Borough,Neighborhood,Latitude,Longitude


In [7]:
# Extract necessary neighborhood information into the dataframe: name, latitude, longitude
for data in zurich_data_features:
    neighborhood_name = data['properties']['name'] 

    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': "Zürich",
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, 
                                           ignore_index=True)
    

In [8]:
neighborhoods.shape

(22, 4)

In [9]:
# How many neighborhoods of Zurich did we get
print('The dataframe has {} neighborhoods.'.format(
        len(neighborhoods['Neighborhood'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 22 neighborhoods.


In [10]:
neighborhoods

,Borough,Neighborhood,Latitude,Longitude
0,Zürich,Hirslanden,47.363057,8.570403
1,Zürich,Witikon,47.359977,8.600887
2,Zürich,Leimbach,47.331480,8.509313
3,Zürich,Wollishofen,47.341491,8.532956
4,Zürich,Enge,47.360931,8.533140
5,Zürich,Riesbach,47.354570,8.556241
6,Zürich,Affoltern,47.423281,8.509221
7,Zürich,Seebach,47.423695,8.542830
8,Zürich,Oerlikon,47.409348,8.545117
9,Zürich,Höngg,47.407904,8.496424


## Visualize the neighborhoods of Zurich on the map

In [11]:
# Get the geographical cooordinates of Zurich
address = 'Zurich, Zurich'

geolocator = Nominatim(user_agent="zh_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Zurich are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of Zurich are 47.3744489, 8.5410422.


In [12]:
# create map of Zurich using latitude and longitude values
map_zh = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, neighborhood, borough in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Neighborhood'], neighborhoods['Borough']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_zh)  
    
map_zh

## Foursquare Credentials and Version

In [13]:
CLIENT_ID = 'D0WVJEV1OWOCGMZ23LW3AV2PUIHF2QXIWCLWQS0V0NBMWV0Q' # your Foursquare ID
CLIENT_SECRET = 'F54CW4QHMTFO23ZZ0LTCEC4XU42PVEEADHNJCNPNDTKVVXIQ' # your Foursquare Secret
ACCESS_TOKEN = 'EUXOMD4V042AJ0VBANQVMRQDPSALDTJLWFD4CVMNIMNYIPVE' # Foursquare Access Token
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('My credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

My credentails:
CLIENT_ID: D0WVJEV1OWOCGMZ23LW3AV2PUIHF2QXIWCLWQS0V0NBMWV0Q
CLIENT_SECRET:F54CW4QHMTFO23ZZ0LTCEC4XU42PVEEADHNJCNPNDTKVVXIQ


In [14]:
#Lets take the first neighborhood in the dataframe
#neighborhoods.loc[0, 'Neighborhood']

In [15]:
#neighborhood_latitude = neighborhoods.loc[0, 'Latitude'] # neighborhood latitude value
#neighborhood_longitude = neighborhoods.loc[0, 'Longitude'] # neighborhood longitude value
#neighborhood_name = neighborhoods.loc[0, 'Neighborhood'] # neighborhood name

#print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
#                                                               neighborhood_latitude, 
#                                                               neighborhood_longitude))

In [16]:
#results = requests.get(url).json()
#results

## Explore neighborhoods in Zurich

In [17]:
# function that extracts the category of the venue
#def get_category_type(row):
#    try:
#        categories_list = row['categories']
#    except:
#        categories_list = row['venue.categories']
        
#    if len(categories_list) == 0:
#        return None
#    else:
#        return categories_list[0]['name']

In [18]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [19]:
#venues = results['response']['groups'][0]['items']
    
#nearby_venues = pd.json_normalize(venues) # flatten JSON

# filter columns
#filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
#nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
#nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
#nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

#nearby_venues.head()

In [20]:
#print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

In [21]:
zurich_venues = getNearbyVenues(names=neighborhoods['Neighborhood'],
                                latitudes=neighborhoods['Latitude'],
                                longitudes=neighborhoods['Longitude']
                                )

Hirslanden
Witikon
Leimbach
Wollishofen
Enge
Riesbach
Affoltern
Seebach
Oerlikon
Höngg
Unterstrass
Wipkingen
Schwamendingen
Oberstrass
Altstetten
Aussersihl
Altstadt
Fluntern
Hottingen
Albisrieden
Wiedikon
Industriequartier


In [22]:
# How many venues in the Zurich neighborhoods did Foursquare return
print(zurich_venues.shape)
zurich_venues.head()

(520, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Hirslanden,47.363057,8.570403,"Marcellino, Pane e Vino",47.360202,8.565569,Mediterranean Restaurant
1,Hirslanden,47.363057,8.570403,Wilder Mann,47.360341,8.566796,Swiss Restaurant
2,Hirslanden,47.363057,8.570403,Ristorante Antiquario,47.360676,8.566550,Italian Restaurant
3,Hirslanden,47.363057,8.570403,Klusplatz,47.364025,8.566751,Plaza
4,Hirslanden,47.363057,8.570403,Bistro Le Puy,47.359382,8.568897,French Restaurant


In [23]:
# Check unique categories we received for Zurich venues
print('There are {} unique categories.'.format(len(zurich_venues['Venue Category'].unique())))

There are 142 unique categories.


In [24]:
zurich_venues

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Hirslanden,47.363057,8.570403,"Marcellino, Pane e Vino",47.360202,8.565569,Mediterranean Restaurant
1,Hirslanden,47.363057,8.570403,Wilder Mann,47.360341,8.566796,Swiss Restaurant
2,Hirslanden,47.363057,8.570403,Ristorante Antiquario,47.360676,8.566550,Italian Restaurant
3,Hirslanden,47.363057,8.570403,Klusplatz,47.364025,8.566751,Plaza
4,Hirslanden,47.363057,8.570403,Bistro Le Puy,47.359382,8.568897,French Restaurant
5,Hirslanden,47.363057,8.570403,VBZ Klusplatz,47.364090,8.566398,Tram Station
6,Hirslanden,47.363057,8.570403,Sonnenberg,47.366290,8.567369,Park
7,Hirslanden,47.363057,8.570403,California House,47.365324,8.564940,Hotel
8,Witikon,47.359977,8.600887,Coop,47.360204,8.594794,Supermarket
9,Witikon,47.359977,8.600887,Glockenacker,47.360889,8.598892,Bus Station


In [25]:
zurich_venues['Venue Category'].unique()

array(['Mediterranean Restaurant', 'Swiss Restaurant',
       'Italian Restaurant', 'Plaza', 'French Restaurant', 'Tram Station',
       'Park', 'Hotel', 'Supermarket', 'Bus Station', 'Soccer Field',
       'Grocery Store', 'Tennis Court', 'Moroccan Restaurant',
       'Restaurant', 'Music Venue', 'Cheese Shop', 'Community Center',
       'Irish Pub', 'Pool', 'Salon / Barbershop', 'Fast Food Restaurant',
       'Gas Station', 'History Museum', 'BBQ Joint', 'Sauna / Steam Room',
       'Thai Restaurant', 'Burger Joint', 'Bar', 'Train Station',
       'Middle Eastern Restaurant', 'Cupcake Shop', 'Chinese Restaurant',
       'Coffee Shop', 'Japanese Restaurant', 'Beach', 'Movie Theater',
       'Café', 'Garden', 'Taverna', 'Seafood Restaurant', 'Bakery',
       'Art Museum', 'Museum', 'Mexican Restaurant', 'Beer Garden',
       'Wine Bar', 'Playground', 'Sushi Restaurant',
       'Arts & Crafts Store', 'Food Court', 'Snack Place',
       'Department Store', 'Athletics & Sports', 'Diner', 

In [26]:
# Let's check now many neighborhoods have train stations
#neigh_with_train = zurich_venues[zurich_venues['Venue Category'] == 'Train Station' ].reset_index(drop=True)
#neigh_with_train

In [27]:
# Narrow the choice of Zurich neighborhoods by considering those with highest number of public transportation (train, tram, bus stations) and supermarkets
#target=zurich_venues.loc[zurich_venues['Venue Category'].isin(['Train Station', 'Tram Station','Bus Station', 'Supermarket'])]   
target=zurich_venues.loc[zurich_venues['Venue Category'].isin(['Train Station', 'Tram Station','Bus Station'])]   
target

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
5,Hirslanden,47.363057,8.570403,VBZ Klusplatz,47.364090,8.566398,Tram Station
9,Witikon,47.359977,8.600887,Glockenacker,47.360889,8.598892,Bus Station
12,Witikon,47.359977,8.600887,VBZ Loorenstrasse,47.359903,8.594463,Bus Station
25,Wollishofen,47.341491,8.532956,VBZ Rote Fabrik,47.342868,8.536043,Bus Station
26,Wollishofen,47.341491,8.532956,VBZ Morgental,47.343597,8.530199,Tram Station
30,Wollishofen,47.341491,8.532956,VBZ Seerose,47.339308,8.537885,Bus Station
52,Enge,47.360931,8.533140,Bahnhof Zürich Enge,47.364305,8.530746,Train Station
56,Enge,47.360931,8.533140,VBZ Bahnhof Enge / Bederstrasse,47.364629,8.530435,Tram Station
97,Riesbach,47.354570,8.556241,VBZ Fröhlichstrasse,47.353632,8.556770,Tram Station
104,Affoltern,47.423281,8.509221,Bahnhof Zürich Affoltern,47.420950,8.507619,Train Station


In [29]:
target.shape
target.insert(7, 'Venue Group', "Transportation", True)
target

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Venue Group
5,Hirslanden,47.363057,8.570403,VBZ Klusplatz,47.364090,8.566398,Tram Station,Transportation
9,Witikon,47.359977,8.600887,Glockenacker,47.360889,8.598892,Bus Station,Transportation
12,Witikon,47.359977,8.600887,VBZ Loorenstrasse,47.359903,8.594463,Bus Station,Transportation
25,Wollishofen,47.341491,8.532956,VBZ Rote Fabrik,47.342868,8.536043,Bus Station,Transportation
26,Wollishofen,47.341491,8.532956,VBZ Morgental,47.343597,8.530199,Tram Station,Transportation
30,Wollishofen,47.341491,8.532956,VBZ Seerose,47.339308,8.537885,Bus Station,Transportation
52,Enge,47.360931,8.533140,Bahnhof Zürich Enge,47.364305,8.530746,Train Station,Transportation
56,Enge,47.360931,8.533140,VBZ Bahnhof Enge / Bederstrasse,47.364629,8.530435,Tram Station,Transportation
97,Riesbach,47.354570,8.556241,VBZ Fröhlichstrasse,47.353632,8.556770,Tram Station,Transportation
104,Affoltern,47.423281,8.509221,Bahnhof Zürich Affoltern,47.420950,8.507619,Train Station,Transportation


In [30]:
target.groupby(["Neighborhood"]).count().sort_values(by=['Venue Category'], ascending=False)

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Venue Group
Neighborhood,,,,,,,
Schwamendingen,5,5,5,5,5,5,5
Wiedikon,4,4,4,4,4,4,4
Unterstrass,4,4,4,4,4,4,4
Oerlikon,3,3,3,3,3,3,3
Wipkingen,3,3,3,3,3,3,3
Wollishofen,3,3,3,3,3,3,3
Albisrieden,2,2,2,2,2,2,2
Enge,2,2,2,2,2,2,2
Seebach,2,2,2,2,2,2,2


In [31]:
target.value_counts(['Neighborhood', 'Venue Category']).reset_index(name='count')

,Neighborhood,Venue Category,count
0,Wiedikon,Bus Station,4
1,Unterstrass,Tram Station,4
2,Schwamendingen,Tram Station,3
3,Oerlikon,Tram Station,2
4,Wollishofen,Bus Station,2
5,Witikon,Bus Station,2
6,Wipkingen,Bus Station,2
7,Seebach,Bus Station,2
8,Schwamendingen,Bus Station,2
9,Affoltern,Train Station,1


In [36]:
# It seems that Schwamedingen, Oerlikon and Wollishofen have highest number of train, tram, bus stations. Lets take food places into consideration too
target = target.append(zurich_venues.loc[zurich_venues['Venue Category'].str.contains("Restaurant")])
target = target.append(zurich_venues.loc[zurich_venues['Venue Category'].str.contains("Café")])
#target = target.append(zurich_venues.loc[zurich_venues['Venue Category'].str.contains("Shop")])
#target = target.append(zurich_venues.loc[zurich_venues['Venue Category'].str.contains("Store")])
#target = target.append(zurich_venues.loc[zurich_venues['Venue Category'].str.contains("Supermarket")])
target

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Venue Group
0,Hirslanden,47.363057,8.570403,VBZ Klusplatz,47.364090,8.566398,Tram Station,Transportation
1,Witikon,47.359977,8.600887,Glockenacker,47.360889,8.598892,Bus Station,Transportation
2,Witikon,47.359977,8.600887,VBZ Loorenstrasse,47.359903,8.594463,Bus Station,Transportation
3,Wollishofen,47.341491,8.532956,VBZ Rote Fabrik,47.342868,8.536043,Bus Station,Transportation
4,Wollishofen,47.341491,8.532956,VBZ Morgental,47.343597,8.530199,Tram Station,Transportation
5,Wollishofen,47.341491,8.532956,VBZ Seerose,47.339308,8.537885,Bus Station,Transportation
6,Enge,47.360931,8.533140,Bahnhof Zürich Enge,47.364305,8.530746,Train Station,Transportation
7,Enge,47.360931,8.533140,VBZ Bahnhof Enge / Bederstrasse,47.364629,8.530435,Tram Station,Transportation
8,Riesbach,47.354570,8.556241,VBZ Fröhlichstrasse,47.353632,8.556770,Tram Station,Transportation
9,Affoltern,47.423281,8.509221,Bahnhof Zürich Affoltern,47.420950,8.507619,Train Station,Transportation


In [44]:
target.loc[target['Venue Group'].isnull(), 'Venue Group'] = 'Food places'
target

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Venue Group
0,Hirslanden,47.363057,8.570403,VBZ Klusplatz,47.364090,8.566398,Tram Station,Transportation
1,Witikon,47.359977,8.600887,Glockenacker,47.360889,8.598892,Bus Station,Transportation
2,Witikon,47.359977,8.600887,VBZ Loorenstrasse,47.359903,8.594463,Bus Station,Transportation
3,Wollishofen,47.341491,8.532956,VBZ Rote Fabrik,47.342868,8.536043,Bus Station,Transportation
4,Wollishofen,47.341491,8.532956,VBZ Morgental,47.343597,8.530199,Tram Station,Transportation
5,Wollishofen,47.341491,8.532956,VBZ Seerose,47.339308,8.537885,Bus Station,Transportation
6,Enge,47.360931,8.533140,Bahnhof Zürich Enge,47.364305,8.530746,Train Station,Transportation
7,Enge,47.360931,8.533140,VBZ Bahnhof Enge / Bederstrasse,47.364629,8.530435,Tram Station,Transportation
8,Riesbach,47.354570,8.556241,VBZ Fröhlichstrasse,47.353632,8.556770,Tram Station,Transportation
9,Affoltern,47.423281,8.509221,Bahnhof Zürich Affoltern,47.420950,8.507619,Train Station,Transportation


In [45]:
target.shape

(525, 8)

In [46]:
target.groupby(["Neighborhood"]).count().sort_values(by=['Venue Category'], ascending=False)

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Venue Group
Neighborhood,,,,,,,
Industriequartier,99,99,99,99,99,99,99
Aussersihl,92,92,92,92,92,92,92
Altstadt,73,73,73,73,73,73,73
Oerlikon,57,57,57,57,57,57,57
Riesbach,40,40,40,40,40,40,40
Enge,29,29,29,29,29,29,29
Schwamendingen,29,29,29,29,29,29,29
Altstetten,28,28,28,28,28,28,28
Unterstrass,19,19,19,19,19,19,19


In [47]:
# It seems that Industriequartier, Aussersihl, Altstadt and Oerlikon are among winners so far. Lets consider supermarkets and stores in each neighborhood too

target = target.append(zurich_venues.loc[zurich_venues['Venue Category'].str.contains("Shop")])
target = target.append(zurich_venues.loc[zurich_venues['Venue Category'].str.contains("Store")])
target = target.append(zurich_venues.loc[zurich_venues['Venue Category'].str.contains("Supermarket")])
target

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Venue Group
0,Hirslanden,47.363057,8.570403,VBZ Klusplatz,47.364090,8.566398,Tram Station,Transportation
1,Witikon,47.359977,8.600887,Glockenacker,47.360889,8.598892,Bus Station,Transportation
2,Witikon,47.359977,8.600887,VBZ Loorenstrasse,47.359903,8.594463,Bus Station,Transportation
3,Wollishofen,47.341491,8.532956,VBZ Rote Fabrik,47.342868,8.536043,Bus Station,Transportation
4,Wollishofen,47.341491,8.532956,VBZ Morgental,47.343597,8.530199,Tram Station,Transportation
5,Wollishofen,47.341491,8.532956,VBZ Seerose,47.339308,8.537885,Bus Station,Transportation
6,Enge,47.360931,8.533140,Bahnhof Zürich Enge,47.364305,8.530746,Train Station,Transportation
7,Enge,47.360931,8.533140,VBZ Bahnhof Enge / Bederstrasse,47.364629,8.530435,Tram Station,Transportation
8,Riesbach,47.354570,8.556241,VBZ Fröhlichstrasse,47.353632,8.556770,Tram Station,Transportation
9,Affoltern,47.423281,8.509221,Bahnhof Zürich Affoltern,47.420950,8.507619,Train Station,Transportation


In [48]:
target.shape

(606, 8)

In [49]:
target.loc[target['Venue Group'].isnull(), 'Venue Group'] = 'Stores'
target

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Venue Group
0,Hirslanden,47.363057,8.570403,VBZ Klusplatz,47.364090,8.566398,Tram Station,Transportation
1,Witikon,47.359977,8.600887,Glockenacker,47.360889,8.598892,Bus Station,Transportation
2,Witikon,47.359977,8.600887,VBZ Loorenstrasse,47.359903,8.594463,Bus Station,Transportation
3,Wollishofen,47.341491,8.532956,VBZ Rote Fabrik,47.342868,8.536043,Bus Station,Transportation
4,Wollishofen,47.341491,8.532956,VBZ Morgental,47.343597,8.530199,Tram Station,Transportation
5,Wollishofen,47.341491,8.532956,VBZ Seerose,47.339308,8.537885,Bus Station,Transportation
6,Enge,47.360931,8.533140,Bahnhof Zürich Enge,47.364305,8.530746,Train Station,Transportation
7,Enge,47.360931,8.533140,VBZ Bahnhof Enge / Bederstrasse,47.364629,8.530435,Tram Station,Transportation
8,Riesbach,47.354570,8.556241,VBZ Fröhlichstrasse,47.353632,8.556770,Tram Station,Transportation
9,Affoltern,47.423281,8.509221,Bahnhof Zürich Affoltern,47.420950,8.507619,Train Station,Transportation


In [55]:
#target.groupby(["Neighborhood"]).count().sort_values(by=['Venue Category'], ascending=False)
target.groupby(["Venue Group", "Neighborhood"]).count().sort_values(by=['Venue Group','Venue Category'], ascending=False)

Neighborhood Latitude  \
Venue Group    Neighborhood                               
Transportation Schwamendingen                         5   
               Unterstrass                            4   
               Wiedikon                               4   
               Oerlikon                               3   
               Wipkingen                              3   
               Wollishofen                            3   
               Albisrieden                            2   
               Aussersihl                             2   
               Enge                                   2   
               Seebach                                2   
               Witikon                                2   
               Affoltern                              1   
               Altstadt                               1   
               Altstetten                             1   
               Hirslanden                             1   
               Hottingen                              1   
               Höngg                                  1   
               Riesbach                               1   
Stores         Altstadt                              24   
               Industriequartier                     13   
               Riesbach                               6   
               Altstetten                             5   
               Oerlikon                               5   
               Wipkingen                              5   
               Enge                                   4   
               Unterstrass                            4   
               Wollishofen                            3   
               Affoltern                              2   
               Fluntern                               2   
               Schwamendingen                         2   
               Witikon                                2   
               Albisrieden                            1   
               Aussersihl                             1   
               Höngg                                  1   
               Wiedikon                               1   
Food places    Industriequartier                     99   
               Aussersihl                            90   
               Altstadt                              72   
               Oerlikon                              54   
               Riesbach                              39   
               Altstetten                            27   
               Enge                                  27   
               Schwamendingen                        24   
               Unterstrass                           15   
               Hirslanden                            12   
               Wollishofen                           12   
               Wipkingen                              6   
               Fluntern                               3   
               Hottingen                              3   
               Leimbach                               3   

                                  Neighborhood Longitude  Venue  \
Venue Group    Neighborhood                                       
Transportation Schwamendingen                          5      5   
               Unterstrass                             4      4   
               Wiedikon                                4      4   
               Oerlikon                                3      3   
               Wipkingen                               3      3   
               Wollishofen                             3      3   
               Albisrieden                             2      2   
               Aussersihl                              2      2   
               Enge                                    2      2   
               Seebach                                 2      2   
               Witikon                                 2      2   
               Affoltern                               1      1   
               Altstadt    

In [ ]:
# Narrow the choice of Zurich neighborhoods by considering those with most number of poblic transportation (train, tram, bus stations) as well as most restaurants and supermarkets
#subset = zurich_venues.loc[zurich_venues['Venue Category'].isin(['Train Station', 'Tram Station','Bus Station','Restaurant', 'Supermarket'])]
#subset

In [ ]:
# Narrow the choice of Zurich neighborhoods by considering only those with train stations
#subset = zurich_venues.loc[zurich_venues['Neighborhood'].isin(['Enge', 'Affoltern','Oerlikon','Aussersihl'])]
#subset

In [ ]:
# Out of the subset we will select the neighborhood with the highest number of restaurants and supermarkets
rest_sup=subset[subset['Venue Category'].str.contains("Restaurant")]
rest_sup

In [ ]:
# How many restaurants are in the selected neighborhoods
print(rest_sup.groupby(['Neighborhood']).size())

In [ ]:
rest_sup=rest_sup.append(subset[subset['Venue Category'].str.contains("Supermarket")])
rest_sup

In [ ]:
# How many restaurants and supermarkets are in the selected neighborhoods with train stations: 'Enge', 'Affoltern','Oerlikon','Aussersihl'
print(rest_sup.groupby(['Neighborhood']).size())

In [ ]:
# Which of these four neighborhoods have most venues - lets sort and print the total list of zurich neighborhoods
zurich_venues.groupby('Neighborhood').count().sort_values(by=['Venue'], ascending=False)

In [ ]:
#res = requests.get(url).json()
#res

In [ ]:
# Search for stores nearby
#search_query = 'store'
#latitude = neigh_with_train.loc[3,'Venue Latitude']
#longitude = neigh_with_train.loc[3,'Venue Longitude']
#url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, ACCESS_TOKEN, VERSION, search_query, radius, LIMIT)
#print(url)

In [ ]:
# assign relevant part of JSON to venues
#stores = res['response']['venues']

# tranform venues into a dataframe
# dataframe = pd.json_normalize(stores)
# dataframe.shape

In [ ]:
# one hot encoding
zurich_onehot = pd.get_dummies(zurich_venues[['Venue Category']], prefix="", prefix_sep="")
#zurich_onehot = pd.get_dummies(neigh_with_train[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
zurich_onehot['Neighborhood'] = zurich_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [zurich_onehot.columns[-1]] + list(zurich_onehot.columns[:-1])
zurich_onehot = zurich_onehot[fixed_columns]

zurich_onehot.head()

In [ ]:
zurich_onehot.shape

In [ ]:
zurich_onehot

In [ ]:
# Group rows by neighborhood and by taking the mean of the frequency of occurrence of each category
zurich_grouped = zurich_onehot.groupby('Neighborhood').mean().reset_index()
zurich_grouped

In [ ]:
zurich_grouped.shape

In [ ]:
num_top_venues = 5

for hood in zurich_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = zurich_grouped[zurich_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')